# Ανίχνευση Clickbait με Stochastic Gradient Descent (SGD)

## 1. Εισαγωγή
Στην παρούσα ενότητα εξετάζουμε την απόδοση ενός γραμμικού ταξινομητή που εκπαιδεύεται μέσω του αλγορίθμου **Stochastic Gradient Descent (SGD)**. Ο SGD αποτελεί μια αποδοτική προσέγγιση για την προσαρμογή γραμμικών μοντέλων (όπως Linear SVM και Logistic Regression) σε μεγάλα σύνολα δεδομένων, καθώς ενημερώνει τα βάρη του μοντέλου επαναληπτικά για κάθε δείγμα ξεχωριστά.

Τα δεδομένα εισόδου είναι διανύσματα UMAP 500 διαστάσεων. Λόγω της φύσης του SGD, ο οποίος είναι ευαίσθητος στην κλίμακα των χαρακτηριστικών, απαιτείται ειδική προεπεξεργασία (Scaling) πριν την εκπαίδευση.

## 2. Μεθοδολογία Πειράματος
Η διαδικασία ακολουθεί αυστηρά πρότυπα για τη διασφάλιση της εγκυρότητας των αποτελεσμάτων:

1.  **Φόρτωση Δεδομένων**: Ανάγνωση των αρχείων Parquet και διαχωρισμός Features/Labels.
2.  **Κανονικοποίηση (Feature Scaling)**: Εφαρμογή **StandardScaler** (Z-score normalization). Αυτό είναι κρίσιμο βήμα για τον SGD, καθώς χαρακτηριστικά με διαφορετικές κλίμακες μπορούν να εμποδίσουν τη σύγκλιση του αλγορίθμου.
3.  **Hyperparameter Tuning (Φάση 1)**: Χρήση του **Optuna** για την εύρεση των βέλτιστων παραμέτρων (`loss`, `penalty`, `alpha`, κ.α.).
4.  **Εκπαίδευση Champion Model (Φάση 2)**: Εκπαίδευση του τελικού μοντέλου στο συνδυαστικό σύνολο (Train + Validation) και καταγραφή του χρόνου εκτέλεσης.
5.  **Τελική Αξιολόγηση**: Δοκιμή στο Test Set και καταγραφή αναλυτικών μετρικών στο **MLflow**.

In [ ]:
import pandas as pd
import numpy as np
import optuna
import mlflow
import sys
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# Ρύθμιση Paths για εντοπισμό των βοηθητικών αρχείων και δεδομένων
# Θεωρούμε ότι το notebook βρίσκεται στο: Models/Stochastic Gradient Decent/
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import mlflow_helper

project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
DATA_FOLDER = os.path.join(project_root, 'data', 'clean', 'umap')

print(f"Project Root: {project_root}")
print(f"Data Folder: {DATA_FOLDER}")

## 3. Φόρτωση και Προετοιμασία Δεδομένων

Χρησιμοποιούμε την ίδια robust συνάρτηση φόρτωσης με τα υπόλοιπα πειράματα, η οποία:
* Διαχειρίζεται αυτόματα αρχεία Parquet διαφορετικών engines (fastparquet/pyarrow).
* Εντοπίζει δυναμικά τις στήλες των χαρακτηριστικών (`umap_...`) και της ετικέτας (`label`, `target`, κλπ.).
* Υποστηρίζει την ανάγνωση labels από εξωτερικά αρχεία CSV αν αυτά λείπουν από το Parquet.

In [ ]:
def load_split_data(data_path):
    files = {
        "Train": "train_umap_500.parquet",
        "Valid": "valid_umap_500.parquet",
        "Test":  "test_umap_500.parquet"
    }

    loaded_data = {}
    possible_label_cols = ['labels', 'label', 'target', 'class', 'is_clickbait']

    print(f"Έναρξη διαδικασίας φόρτωσης από: {data_path}")

    for name, filename in files.items():
        file_path = os.path.join(data_path, filename)

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Το αρχείο {filename} δεν βρέθηκε.")

        try:
            df = pd.read_parquet(file_path, engine='fastparquet')
        except:
            df = pd.read_parquet(file_path, engine='pyarrow')

        # 1. Εντοπισμός Features
        feature_cols = [c for c in df.columns if c.startswith("umap_")]
        if not feature_cols:
            feature_cols = [c for c in df.columns if c not in possible_label_cols]

        # 2. Εντοπισμός Labels
        label_col = None
        for col in possible_label_cols:
            if col in df.columns:
                label_col = col
                break

        # Fallback: Αναζήτηση εξωτερικού αρχείου ή αυτόματος εντοπισμός
        if label_col is None:
            remaining = [c for c in df.columns if c not in feature_cols]
            if len(remaining) == 1:
                label_col = remaining[0]
                print(f"   [{name}] Label column autodetected: '{label_col}'")
            else:
                prefix = filename.split('_')[0]
                ext_path = os.path.join(data_path, f"{prefix}_labels.csv")
                if os.path.exists(ext_path):
                    print(f"   [{name}] Labels loaded from external file: {prefix}_labels.csv")
                    df_labels = pd.read_csv(ext_path)
                    y = df_labels.iloc[:, 0].values.astype(int)
                    X = df[feature_cols].values.astype(np.float32)
                    loaded_data[name] = (X, y)
                    print(f"   [{name}] Loaded Successfully: X={X.shape}, y={y.shape}")
                    continue
                else:
                    raise ValueError(f"Σφάλμα στο {name}: Δεν βρέθηκε label.")

        if label_col:
            if label_col in feature_cols:
                feature_cols.remove(label_col)
            y = df[label_col].values.astype(int)

        X = df[feature_cols].values.astype(np.float32)

        if len(X) != len(y):
             raise ValueError(f"Ασυμφωνία διαστάσεων στο {name}: X={len(X)}, y={len(y)}")

        loaded_data[name] = (X, y)
        print(f"   [{name}] Loaded Successfully: X={X.shape}, y={y.shape}")

    return loaded_data["Train"], loaded_data["Valid"], loaded_data["Test"]

# Εκτέλεση Φόρτωσης
(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_split_data(DATA_FOLDER)

## 4. Κανονικοποίηση Δεδομένων (Scaling)

Η κανονικοποίηση είναι απαραίτητη προϋπόθεση για την ορθή λειτουργία του SGD. Εφαρμόζουμε **StandardScaler** ώστε κάθε χαρακτηριστικό να έχει μέση τιμή 0 και τυπική απόκλιση 1.

**Σημαντικό:**
* Ο scaler εκπαιδεύεται (`fit`) **μόνο** στο Train Set για να αποφευχθεί η διαρροή πληροφορίας (data leakage).
* Οι παράμετροι του scaler εφαρμόζονται (`transform`) στη συνέχεια στα Validation και Test sets.

In [ ]:
print("Εφαρμογή StandardScaler στα δεδομένα...")

# Fit μόνο στο Train set
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Transform στα υπόλοιπα sets με βάση τις παραμέτρους του Train
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

print("Scaling ολοκληρώθηκε επιτυχώς.")
print(f"Mean of first feature (Train): {X_train[:, 0].mean():.4f}, Std: {X_train[:, 0].std():.4f}")

## 5. Βελτιστοποίηση Υπερπαραμέτρων (Optuna Objective)

Η συνάρτηση `objective` διερευνά τον χώρο υπερπαραμέτρων του SGD Classifier. Οι κυριότερες παράμετροι προς βελτιστοποίηση είναι:

* **Loss Function (`loss`)**: Καθορίζει το είδος του μοντέλου (π.χ. `hinge` για Linear SVM, `log_loss` για Logistic Regression).
* **Regularization (`penalty`)**: Επιλογή μεταξύ L1 (Lasso), L2 (Ridge) ή ElasticNet.
* **Alpha (`alpha`)**: Η ισχύς της ποινής (regularization strength). Μικρότερες τιμές οδηγούν σε πιο σύνθετα μοντέλα (κίνδυνος overfitting), ενώ μεγαλύτερες σε πιο απλά (κίνδυνος underfitting).

Επιπλέον, καταγράφουμε τον **χρόνο εκπαίδευσης** για κάθε συνδυασμό παραμέτρων.

In [ ]:
def objective(trial, X_tr, y_tr, X_v, y_v):
    # Χώρος αναζήτησης υπερπαραμέτρων
    loss_type = trial.suggest_categorical("loss", ["hinge", "log_loss", "modified_huber", "perceptron"])
    penalty = trial.suggest_categorical("penalty", ["l2", "l1", "elasticnet"])
    alpha = trial.suggest_float("alpha", 1e-6, 1e-1, log=True)

    params = {
        "loss": loss_type,
        "penalty": penalty,
        "alpha": alpha,
        "max_iter": 1000,
        "early_stopping": True,
        "n_iter_no_change": 5,
        "random_state": 42
    }

    if penalty == "elasticnet":
        params["l1_ratio"] = trial.suggest_float("l1_ratio", 0.0, 1.0)

    model = SGDClassifier(**params)

    # Μέτρηση Χρόνου Εκπαίδευσης
    start_time = time.time()
    model.fit(X_tr, y_tr)
    training_time = time.time() - start_time

    # Αξιολόγηση
    preds = model.predict(X_v)
    f1 = f1_score(y_v, preds)
    acc = accuracy_score(y_v, preds)

    metrics = {
        "val_f1": f1,
        "val_accuracy": acc,
        "training_time_sec": training_time
    }

    # Καταγραφή στο MLflow
    mlflow_helper.log_optuna_trial(
        trial,
        params,
        metrics,
        model,
        run_name_prefix="SGD_Trial"
    )

    return f1

## 6. Εκτέλεση Πειράματος & Τελική Αξιολόγηση

Η πειραματική διαδικασία χωρίζεται σε δύο φάσεις στο MLflow:
1.  **Hyperparameter Tuning Run**: Διεξαγωγή 20 δοκιμών (trials) για την εύρεση των βέλτιστων παραμέτρων.
2.  **Champion Model Run**: Εκπαίδευση του τελικού μοντέλου με τις βέλτιστες παραμέτρους στο ενιαίο σύνολο (Train + Validation) και αξιολόγηση στο Test Set.

Στο τελικό στάδιο, παράγονται και αποθηκεύονται γραφήματα όπως Confusion Matrix, ROC Curve και Precision-Recall Curve.

In [ ]:
EXPERIMENT_NAME = "Clickbait_SGD_UMAP_Final"
mlflow_helper.setup_mlflow(EXPERIMENT_NAME)

print(f"\nΈναρξη Πειράματος: {EXPERIMENT_NAME}")

# --- ΦΑΣΗ 1: Hyperparameter Tuning ---
print("\n[ΦΑΣΗ 1] Αναζήτηση Βέλτιστων Παραμέτρων (Optuna)...")

with mlflow.start_run(run_name="SGD_Hyperparameter_Tuning") as tuning_run:
    mlflow.log_param("dataset", "UMAP_500")
    mlflow.log_param("scaling", "StandardScaler")

    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="maximize", sampler=sampler)

    # Εκτέλεση 20 δοκιμών
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=20)

    print(f"Best Params found: {study.best_params}")
    print(f"Best Val F1: {study.best_value:.4f}")

# --- ΦΑΣΗ 2: Champion Model Training ---
print("\n[ΦΑΣΗ 2] Εκπαίδευση & Αποθήκευση Champion Model...")

with mlflow.start_run(run_name="SGD_Champion_Model") as final_run:

    # Ανάκτηση και ρύθμιση βέλτιστων παραμέτρων
    best_params = study.best_params
    best_params.update({
        "max_iter": 1000,
        "early_stopping": True,
        "n_iter_no_change": 5,
        "random_state": 42
    })

    mlflow.log_params(best_params)
    mlflow.log_param("model_type", "SGD_Champion")

    final_model = SGDClassifier(**best_params)

    # Ένωση scaled συνόλων για την τελική εκπαίδευση
    X_full_train = np.concatenate((X_train, X_val))
    y_full_train = np.concatenate((y_train, y_val))

    # Εκπαίδευση και χρονομέτρηση
    start_t = time.time()
    final_model.fit(X_full_train, y_full_train)
    final_train_time = time.time() - start_t
    print(f"Total Training Time: {final_train_time:.2f} sec")

    # Αποθήκευση μοντέλου
    mlflow.sklearn.log_model(final_model, artifact_path="champion_model")

    # Τελική Αξιολόγηση στο Test Set
    print("Υπολογισμός τελικών μετρικών στο Test Set...")
    mlflow_helper.evaluate_and_log_metrics(
        final_model,
        X_test,
        y_test,
        prefix="test",
        training_time=final_train_time
    )

    print(f"\nΟλοκληρώθηκε επιτυχώς. Run ID: {final_run.info.run_id}")
    print(f"Αποτελέσματα διαθέσιμα στο MLflow Run: 'SGD_Champion_Model'")